In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

In [57]:
import difflib
from ipywidgets import HTML
    
def display_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=red>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=blue>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return HTML(''.join(output))

In [2]:
df = pd.read_excel('./data/artist_list_manual_cleaning_map.xlsx', index_col='id')
display(df)

,name,clean name,no_exhibitions,indicative,checked
id,,,,,
6,-,NaN,3429,NaN,1.0
114,Sugár János,Sugár János,124,NaN,1.0
1032,Szombathy Bálint,Szombathy Bálint,119,NaN,1.0
2351,Muladi Brigitta,Muladi Brigitta,112,NaN,1.0
301,Koronczi Endre,Koronczi Endre,108,NaN,1.0
...,...,...,...,...,...
21,Barta István,Barta István,1,NaN,NaN
17,borgó,Borgó,1,NaN,NaN
9,mjbkjhb,NaN,1,NaN,NaN


In [39]:
from ipywidgets import Label, IntProgress, Button, Layout, HBox
from IPython.display import display

In [30]:
def find_fuzzy_variations(df, start=0, end=-1):
    end = len(df) if end<0 else end
    progressbar = IntProgress(min=0, max=100)
    progresslabel = Label("0.000%")
    display(progressbar)
    display(progresslabel)
    stack = set()
    for i in range(start, end):
        A = df.iloc[i]
        for j in range(i+1, len(df)):
            B = df.iloc[j]
            try:
                ratio = fuzz.ratio(A['clean name'], B['clean name'])
                if ratio>95 and ratio<100:
                    yield df.index[i], A['clean name'], df.index[j], B['clean name'], ratio
            except TypeError:
                pass
            
        progress = (i+1-start) / (end-start) 
        progressbar.value = progress*100
        progresslabel.value = "{} {} | {}/{} | {:.3f}%".format(
            i, A['clean name'],
            i+1-start, end-start,
            progress * 100)


In [58]:
with open("./data/fuzzy_map.csv", mode='w', encoding='utf=8') as file:
    file.write('id'+'+\t'+'name'+'\t'+'matchname'+'\t'+'idmatch'+'\t'+'ratio'+'\n')
            
    for match in find_fuzzy_variations(df, start=202, end=210):
        nameid, name, idmatch, matchname, ratio = match
        line = "{}\t{}\t{}\t{}\t{:.2f}\n".format(nameid, name, idmatch, matchname, ratio)
        file.write(line)
        display( HBox([Label(name), display_diff(name, matchname), Label("{}%".format(ratio))]))
        

IntProgress(value=0)

Label(value='0.000%')

KeyboardInterrupt: 